### Future-Features
- [ ] Make a provision for self completion status and parent completion status; 
   If parent is completed - whole subtask is completed; 
   But if user sets the parent task uncomplete afterwards, set the completion status to what was self completion status; 
   Use the OR function between the self and parent completion status to achieve this.

- [ ] Add a `progress` meter that increases with subtask completion

- [ ] Add `Repeats` class provisionality

- [ ] Implement `Deadlines` class properly

In [1]:
# Imports

import time, math
import numpy as np
from enum import Enum, auto

In [2]:
# Global Variables

max_weight = 5

file_logs = "Logs.txt"

class Deadlines(Enum):
    no_deadline = auto()
    date_deadline = auto()
    duration_deadline = auto()

# Removed time_deadline
# Shifted repeat_deadline to be its own thing independent of Deadlines - thus utilizing Repeats class

class Repeats(Enum):
    endless = auto()
    till_date = auto()
    repeat_count = auto()

In [3]:
class Common_Functions:
    def log(self, message: str, display: bool) -> None:
        timestamp = time.strftime("%I:%M:%S %p, %d %b %Y")
        log_message = f"({timestamp}) {self.name}: {message}"

        with open(file_logs, "a") as file:
            file.write(log_message)

        if display:
            print(log_message)

    def log_error(self, error_type, error_message: str) -> None:
        self.log(error_message, False)
        raise error_type(error_message)

In [15]:
class Task(Common_Functions):
    '''
    Instantiate a new `Task` by provding its to-do title.
    ### Required Parameters
    `name`: Set the work-title you have to do

    ### Additional Parameters
    `details` (_str_): Add a description of the task you are doing
    `weight` (_float_): Add a *relative* priority value to the task from a range of values
    `deadline_type` (_Deadlines_): 
    '''
    task_number = 1 # Used to make the ID of the current task
    tasks_generated = 1 # Stores the number of tasks ever generated
    tasks_done = 0 # Stores the number of tasks ever done

    def __init__(self, name) -> None:
        self.name = name
        self.ID = self.task_number
        self.details = ""
        self.weight = 1
        self.deadline_type = Deadlines.no_deadline
        self.completed = False

        self.parent = None
        self.subtasks = np.array([])

        Task.task_number += 1
        Task.tasks_generated += 1

        self.log(f"Instantiated the task")

    def set_description(self, details: str) -> None:
        self.details = details
        self.log(f"Modified description")

    def set_priority(self, weight: float) -> None:
        if self.weight > max_weight:
            self.log_error(ValueError, f"Weight can't be greater than {max_weight}!")

        self.weight = weight
        self.log("Modified Weight")

    def set_deadline(self, deadline_type, deadline_info) -> None:
        match deadline_type:
            case Deadlines.date_deadline.value:
                self.deadline_type = Deadlines.date_deadline
            case Deadlines.time_deadline.value:
                self.deadline_type = Deadlines.time_deadline
            case Deadlines.duration_deadline.value:
                self.deadline_type = Deadlines.duration_deadline
            case Deadlines.repeat_deadline.value:
                self.deadline_type = Deadlines.date_deadline
            case _:
                self.deadline_type = Deadlines.no_deadline

    def add_subtask(self, subtask_name: str) -> None:
        new_subtask = Task(subtask_name)
        self.set_subtask(new_subtask)

    def set_subtask(self, subtask) -> None:
        if not isinstance(subtask, Task):
            self.log_error(TypeError, f"Can't make a non-Task object as sub-task!")

        self.subtasks = self.subtasks.append(self.subtasks, subtask)
        self.log(f"Added {subtask.name} as a subtask")

        subtask.parent = self
        subtask.log(f"Set {self.name} as a parent")

    def set_completion_status(self, status: bool) -> None:
        if not isinstance(status, bool):
            self.log_error(ValueError, f"Can't set the completion status to {status}! (should be True/False)")
        
        previous_status = self.completed
        self.completed = status
        self.log(f"Set completion status to {status}")

        # Set subtasks to complete automatically as well
        if previous_status ^ status and self.subtasks.size != 0:
            batch_complete_subtasks = np.vectorize(lambda subtask: subtask.set_completion_status(status))
            completion = batch_complete_subtasks(self.subtasks)
            self.log(f"Set all subtask's completion status to {status}")
        
        # Check if parent's all substasks are completed
        if (not (self.parent is None)) and self.parent.check_subtasks_completion():
            self.parent.set_completion_status(True)
            self.parent.log(f"Set completion status to True (all subtasks are completed)")

    def check_subtasks_completion(self) -> bool:
        if self.subtasks.size == 0:
            return False
        
        # "Vectorizes" the function which returns the completion status of subtasks
        # "Vectorization" of a function means that it can input an array of scalar instead of one scalar as usual (and thus outputs an array of scalars) 
        get_subtask_status = np.vectorize(lambda task: task.completed) # A function
        subtask_completions = get_subtask_status(self.subtasks)

        return True if np.all(subtask_completions == True) else False

#### Testing classes and functioning of production-code

In [8]:
task1 = Task("Complete CS201 Quiz 1")
print(task1.ID)
task1.ID = 3
print(task1.ID)
task1.deadline_type

1
3


<Deadlines.no_deadline: 1>

In [9]:
task2 = Task("Go to Class - HS201")
print(task2.ID)
task1.ID = 10
print(task2.ID)
Task.ID = 1
print(task2.ID)

2
2
2


#### Testing Modules and Functions for implementation

In [10]:
class Point:
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y

    def __repr__(self) -> str:
       return f"{self.__class__.__name__}({self.x}, {self.y})"
    
    def scale(self, factor: int | float):
        # print(not isinstance(factor, int), not isinstance(factor, float))
        # if (not isinstance(factor, int)) or (not isinstance(factor, float)):
        #     print("Can't scale a point by non-number quantity!")
        #     return None

        return Point(self.x * factor, self.y * factor)

In [11]:
p1 = Point(2, 3)
p1.scale("1")

Point(11, 111)

In [12]:
print(True ^ True) # False
print(True ^ False) # True
print(False ^ True) # True
print(False ^ False) # False

False
True
True
False
